In [ ]:
from joblib import load
regressor=load('/content/drive/MyDrive/Colab Notebooks/Untitled folder/Scenario9/Scenario9/regressor.pkl')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout,GRU
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

import cv2
import json
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
X_=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Untitled folder/Scenario9/val_data_sequence.csv',dtype=np.float64,header=None)
X_t=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Untitled folder/Scenario9/train_data_sequence.csv',dtype=np.float64,header=None)

In [ ]:
X=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Untitled folder/result/bboxscene23.csv',dtype=np.float64,header=None)
Y_=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Untitled folder/Scenario9/Scenario9/development_dataset/scenario9_dev_test.csv')
Y=Y_['beam_index_1']

X=X.iloc[:-1]
X=X.drop([0])
Y.index=X.index

In [ ]:
X

,0,1,2,3
1,0.261879,0.576826,0.308344,0.178867
2,0.566934,0.578812,0.301383,0.182042
3,0.238123,0.580782,0.318684,0.185612
4,0.757559,0.501874,0.188530,0.124087
5,0.517090,0.580098,0.302266,0.182195
...,...,...,...,...
589,0.000000,0.000000,0.000000,0.000000
590,0.685925,0.572862,0.300077,0.181474
591,0.274805,0.582007,0.318017,0.187783
592,0.344580,0.577623,0.304445,0.182317


In [ ]:
X=X_.iloc[:,:4]
Y=pd.DataFrame(np.argmax(X_.iloc[:,4:],axis=1))

In [ ]:
pwr_v=X_.iloc[:,4:]

In [ ]:
l=6
Y1=pd.DataFrame(np.zeros([len(Y),l]))
for i in range(l,len(Y)):
    Y1.iloc[i,:]=Y.iloc[i-l:i].transpose()
Y1.index=Y.index
X1=pd.concat([X.iloc[:,:l],Y1],axis=1)


In [ ]:
Y1

,0,1,2,3,4,5
0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
1169,54.0,57.0,58.0,59.0,61.0,61.0
1170,57.0,58.0,59.0,61.0,61.0,61.0
1171,58.0,59.0,61.0,61.0,61.0,61.0
1172,59.0,61.0,61.0,61.0,61.0,61.0


In [ ]:
predicted=pd.DataFrame(np.float64(regressor.predict(X1)))

37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step


In [ ]:
Z=pd.DataFrame(np.zeros([len(predicted),20]))
for i in range(0,len(predicted)):
    Z.iloc[i]=pd.DataFrame(predicted).iloc[i].nlargest(20).index

In [ ]:
Z

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.0,13.0,2.0,1.0,45.0,12.0,27.0,53.0,61.0,4.0,56.0,60.0,26.0,14.0,41.0,57.0,35.0,24.0,37.0,55.0
1,0.0,13.0,2.0,1.0,45.0,12.0,4.0,61.0,14.0,53.0,24.0,60.0,27.0,37.0,56.0,41.0,26.0,3.0,57.0,35.0
2,0.0,13.0,1.0,2.0,12.0,45.0,4.0,14.0,61.0,24.0,53.0,60.0,3.0,37.0,27.0,11.0,56.0,26.0,41.0,57.0
3,0.0,1.0,2.0,13.0,12.0,4.0,45.0,14.0,3.0,61.0,11.0,24.0,37.0,60.0,5.0,53.0,26.0,15.0,27.0,10.0
4,0.0,1.0,2.0,13.0,12.0,4.0,3.0,14.0,5.0,45.0,61.0,11.0,15.0,37.0,26.0,60.0,50.0,10.0,53.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1169,60.0,61.0,59.0,57.0,8.0,49.0,45.0,58.0,55.0,25.0,53.0,27.0,56.0,4.0,0.0,12.0,11.0,9.0,31.0,1.0
1170,60.0,61.0,49.0,59.0,45.0,8.0,57.0,55.0,58.0,25.0,0.0,4.0,27.0,53.0,11.0,12.0,31.0,1.0,56.0,9.0
1171,60.0,61.0,49.0,45.0,59.0,8.0,57.0,55.0,0.0,25.0,4.0,58.0,27.0,11.0,1.0,12.0,53.0,31.0,5.0,9.0
1172,0.0,13.0,12.0,1.0,2.0,11.0,14.0,4.0,61.0,60.0,45.0,5.0,27.0,10.0,24.0,49.0,3.0,25.0,37.0,21.0


In [ ]:
Y=Y.drop([1],axis=0)

In [ ]:
def top_k_accuracy(k,T,y):
    y.index=T.index
    R=T[0]
    A=R==y[0]
    for j in range(0,k):
       for i in T.index:
          if (A[i]!=True):
              A[i]=T.iloc[i,j]==y.loc[i]
    acc=(pd.DataFrame(A).value_counts()[True])/len(T)
    return   acc*100

In [ ]:
pwr_=Y_['unit1_pwr_1']
B=pd.read_csv(pwr_[10],header=None)
pwr=pd.DataFrame(np.zeros([len(pwr_),len(B)]))
for i in range(0,len(pwr)):
  pwr.iloc[i]=pd.read_csv(pwr_[i],engine='python',header=None,encoding='utf-8').transpose()
pwr=pd.DataFrame(pwr)


In [ ]:
V=np.zeros([len(pwr),1])
for i in range(0,len(pwr)):
  V[i]=np.argmax(pwr.iloc[i])
V=pd.DataFrame(V).drop([0],axis=0)

In [ ]:
H=11
accu=np.zeros([H,1])
for i in range(1,H):
  accu[i]=top_k_accuracy(i,Z+1,V)
pd.DataFrame(accu)

ValueError: Length mismatch: Expected axis has 592 elements, new values have 1174 elements

In [ ]:
Z.iloc[0,2]

In [ ]:
o=Z.iloc[0,2]

In [ ]:
pwr=pd.DataFrame(pwr)

In [ ]:
pwr.iloc[1,int(o)]

In [ ]:
pwr.iloc[1,5]

In [ ]:
def power_predict(Z,K,pwr):
  pred_pwr=pd.DataFrame(np.zeros([len(Z),K]))
  pred_pwr.index=Z.index
  pred_pwr.columns=Z.columns[:K]
  pwr=pwr.iloc[1:]
  for i in range(0,len(Z)):
    for j in range(0,K):
      pred_pwr.iloc[i,j]=pwr.iloc[i,int(Z.iloc[i,j]+1)]
  return pred_pwr

In [ ]:
pred_pwr_1=power_predict(Z,1,pwr)
pred_pwr_2=power_predict(Z,2,pwr)
pred_pwr_3=power_predict(Z,3,pwr)
pred_pwr_4=power_predict(Z,4,pwr)
pred_pwr_5=power_predict(Z,5,pwr)
pred_pwr_6=power_predict(Z,6,pwr)
pred_pwr_7=power_predict(Z,7,pwr)
pred_pwr_8=power_predict(Z,8,pwr)
pred_pwr_9=power_predict(Z,9,pwr)
pred_pwr_10=power_predict(Z,10,pwr)
pred_pwr_11=power_predict(Z,11,pwr)
pred_pwr_12=power_predict(Z,12,pwr)
pred_pwr_13=power_predict(Z,13,pwr)
pred_pwr_14=power_predict(Z,14,pwr)
pred_pwr_15=power_predict(Z,15,pwr)
pred_pwr_16=power_predict(Z,16,pwr)
pred_pwr_17=power_predict(Z,17,pwr)
pred_pwr_18=power_predict(Z,18,pwr)
pred_pwr_19=power_predict(Z,19,pwr)
pred_pwr_20=power_predict(Z,20,pwr)


In [ ]:
def pwr_diff(A,pwr,pred_pwr):
  diff=pd.DataFrame(np.zeros([len(A)]))
  for i in range(0,len(A)):
    for j in range(0,len(pred_pwr.columns)):
      diff.iloc[i]=pwr.iloc[i,A.iloc[i]]-max(pred_pwr.iloc[i])
  return diff.iloc[1:len(diff)-1]

In [ ]:
diff_1=pwr_diff(V,pwr,pred_pwr_1)
diff_2=pwr_diff(V,pwr,pred_pwr_2)
diff_3=pwr_diff(V,pwr,pred_pwr_3)
diff_4=pwr_diff(V,pwr,pred_pwr_4)
diff_5=pwr_diff(V,pwr,pred_pwr_5)
diff_6=pwr_diff(V,pwr,pred_pwr_6)
diff_7=pwr_diff(V,pwr,pred_pwr_7)
diff_8=pwr_diff(V,pwr,pred_pwr_8)
diff_9=pwr_diff(V,pwr,pred_pwr_9)
diff_10=pwr_diff(V,pwr,pred_pwr_10)
diff_11=pwr_diff(V,pwr,pred_pwr_11)
diff_12=pwr_diff(V,pwr,pred_pwr_12)
diff_13=pwr_diff(V,pwr,pred_pwr_13)
diff_14=pwr_diff(V,pwr,pred_pwr_14)
diff_15=pwr_diff(V,pwr,pred_pwr_15)
diff_16=pwr_diff(V,pwr,pred_pwr_16)
diff_17=pwr_diff(V,pwr,pred_pwr_17)
diff_18=pwr_diff(V,pwr,pred_pwr_18)
diff_19=pwr_diff(V,pwr,pred_pwr_19)
diff_20=pwr_diff(V,pwr,pred_pwr_20)

In [ ]:
diff=pd.concat([diff_1,diff_2,diff_3,diff_4,diff_5,diff_6,diff_7,diff_8,diff_9,diff_10,diff_11,diff_12,diff_13,diff_14,diff_15,diff_16,diff_17,diff_18,diff_19,diff_20],axis=1)
diff.columns=range(0,len(diff.columns))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(10,6))
fig, (ax1, ax2) = plt.subplots(2, 1)
#predi_loss = predi[['accuracy','val_accuracy']]
plt.title("")
sns.lineplot(np.mean(diff.iloc[:,:10],axis=0));
ax1.plot(np.mean(diff.iloc[:,:10]**2,axis=0))
ax1.set_title('mean squared diffrence of power of K predicted beam index and optimal index')
ax2.plot(np.mean(diff.iloc[:,:10],axis=0))
ax2.set_title('mean diffrence of power of K predicted beam index and optimal index')
plt.subplots_adjust(hspace=0.4)
plt.xlabel('X Axis Label')
plt.ylabel('power')


In [ ]:
def rsp(pred_pwr,K):
  SNR=np.zeros([len(pred_pwr),1])
  for j in range(0,len(pred_pwr)):
    SNR[j]=max(pred_pwr.iloc[j,:K])
  return pd.DataFrame(SNR)

In [ ]:
SNR_1=rsp(pred_pwr,1)
SNR_2=rsp(pred_pwr,2)
SNR_3=rsp(pred_pwr,3)
SNR_4=rsp(pred_pwr,4)
SNR_5=rsp(pred_pwr,5)
SNR_6=rsp(pred_pwr,6)
SNR_7=rsp(pred_pwr,7)
SNR_8=rsp(pred_pwr,8)
SNR_9=rsp(pred_pwr,9)
SNR_10=rsp(pred_pwr,10)


In [ ]:
SNR=pd.concat([SNR_1,SNR_2,SNR_3,SNR_4,SNR_5,SNR_6,SNR_7,SNR_8,SNR_9,SNR_10],axis=1)
SNR.columns=range(1,len(SNR.columns)+1)

In [ ]:
SNR_ratio=pd.DataFrame(np.zeros([len(SNR),len(SNR.columns)]))
SNR.columns=pwr.iloc[:,:10].columns
for i in range(0,len(pwr)-1):
  SNR_ratio.iloc[i]=SNR.iloc[i]/max(pwr.iloc[i])

In [ ]:
SNR_ratio

In [ ]:
G=np.zeros([len(V)])
for i in range(0,len(V)):
  G[i]=np.argmax(pwr.iloc[i])-V.iloc[i]

In [ ]:
np.savetxt('signalpwr.csv',np.mean(SNR,axis=0),delimiter=',')
np.savetxt('signalpwrratio.csv',np.mean(SNR_ratio,axis=0),delimiter=',')

In [ ]:
plt.figure(figsize=(10,6))
#predi_loss = predi[['accuracy','val_accuracy']]
plt.title("Recived SNR for various values of K")
sns.lineplot(np.mean(SNR,axis=0));
plt.xlabel('K')
plt.ylabel('SNR')

In [ ]:
plt.figure(figsize=(10,6))
#predi_loss = predi[['accuracy','val_accuracy']]
plt.title("Recived signal power ratio for various values of K")
sns.lineplot(np.mean(SNR_ratio,axis=0));
#plt.xlabel('K')
plt.ylabel('Recived signal power')